In [ ]:
%load_ext autoreload

In [ ]:
import altair as alt
import polars as pl

from altair_utils import color_value_transit_teal

In [ ]:
quarterly_df = (
    pl.read_csv(
        r"Q:\Data\PeerAgencyDashboards\SFCity-EconRecovery\office_space_vacancy-quarterly-updated250123.csv"
    )
    .select(
        (pl.col("San Francisco").str.strip_chars_end("%").cast(int) / 100).alias(
            "office space vacancy rate"
        ),
        date=pl.col("Date").str.to_date("%b %y"),
    )
    .with_columns(
        year=pl.col("date").dt.year(),
        quarter=pl.col("date").dt.quarter(),
    )
).filter(pl.col("year") > 2006)

In [ ]:
chart = (
    alt.Chart(quarterly_df)
    .mark_line()
    .encode(
        x=alt.X("date"),  # .axis(values=["{y}-1-1" for y in range(2007, 2026, 3)]),
        y=alt.Y("office space vacancy rate").axis(format="%"),
        color=color_value_transit_teal,
        tooltip=[
            "year",
            "quarter",
            alt.Tooltip("office space vacancy rate", format=".0%"),
        ],
    )
)
quarterly_df.write_csv("output/data/office_vacancy.csv")
chart.save("output/Links/office_vacancy.png", scale_factor=3)
chart.properties(width="container").save("output/Links/office_vacancy.html")
chart